In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[["Gender_Male","Age",'EstimatedSalary']]
dep=dataset[["Purchased"]]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
param_grid={'priors':[None], 'var_smoothing': [1e-09,0]}
grid= GridSearchCV(GaussianNB(), param_grid, refit= True, verbose=3, n_jobs=-1, scoring= 'f1_weighted' )
grid.fit(X_train,y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


C:\Anaconda3run\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=GaussianNB(), n_jobs=-1,
             param_grid={'priors': [None], 'var_smoothing': [1e-09, 0]},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[81,  4],
       [ 8, 41]], dtype=int64)

In [13]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [14]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.91      0.95      0.93        85
           1       0.91      0.84      0.87        49

    accuracy                           0.91       134
   macro avg       0.91      0.89      0.90       134
weighted avg       0.91      0.91      0.91       134



In [15]:
from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
roc_score

0.9608643457382953

In [16]:
print ( " the best parameter is ", grid.best_params_)

 the best parameter is  {'priors': None, 'var_smoothing': 1e-09}


In [17]:
grid.cv_results_

{'mean_fit_time': array([0.01356511, 0.01335669]),
 'std_fit_time': array([0.00495167, 0.00249858]),
 'mean_score_time': array([0.0187489 , 0.01914768]),
 'std_score_time': array([0.00171509, 0.00097542]),
 'param_priors': masked_array(data=[None, None],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_var_smoothing': masked_array(data=[1e-09, 0],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'priors': None, 'var_smoothing': 1e-09},
  {'priors': None, 'var_smoothing': 0}],
 'split0_test_score': array([0.82209189, 0.82209189]),
 'split1_test_score': array([0.86875226, 0.86875226]),
 'split2_test_score': array([0.7911521, 0.7911521]),
 'split3_test_score': array([0.96226415, 0.96226415]),
 'split4_test_score': array([0.98101383, 0.98101383]),
 'mean_test_score': array([0.88505485, 0.88505485]),
 'std_test_score': array([0.07512288, 0.07512288]),
 'rank_test_score': array([1, 1])}

In [18]:
re=grid.cv_results_

In [19]:
table= pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_priors,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013565,0.004952,0.018749,0.001715,None,0.0,"{'priors': None, 'var_smoothing': 1e-09}",0.822092,0.868752,0.791152,0.962264,0.981014,0.885055,0.075123,1
1,0.013357,0.002499,0.019148,0.000975,None,0,"{'priors': None, 'var_smoothing': 0}",0.822092,0.868752,0.791152,0.962264,0.981014,0.885055,0.075123,1


In [20]:
grid.best_score_

0.8850548464404348

In [23]:
pre_input= sc.transform([[0,35,60000]])
pre_input

C:\Anaconda3run\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-0.99250926, -0.27290378, -0.27655878]])

In [24]:
result= grid.predict(pre_input)
result

array([0], dtype=int64)